## Create Geometry

In [ ]:
import numpy as np
import nibabel as nib
# import itk
import json
import yaml

## Lung mask

In [ ]:
atm = nib.load('/home/anna/Desktop/ATM_data/ATM_257/ATM_257_0000_seg_lobes.nii.gz')
atm_data = atm.get_fdata().astype(np.uint8)

In [ ]:
for i in range(1, 6):
    lobe = np.where(atm_data == i, 1, 0).astype(np.uint8)
    np.save(f'atm257_{i}.npy', lobe)
    # itk.imwrite(itk.GetImageFromArray(lobe.astype(np.uint8)), f'atm257_{i}.nrrd')

## Bronchi roots

In [ ]:
scan = 'ATM_257'
out_file = f'/home/anna/Desktop/MA/ATM_data/{scan}/{scan}_roots.yml'

lobes = {}
# weird_spacing
with open(f'/home/anna/Desktop/MA/ATM_data/{scan}/{scan}_spacing.yml') as f:
    spacing = yaml.safe_load(f)

p_min = np.array((spacing['x_min'], spacing['y_min'], spacing['z_min']))
p_max = np.array((spacing['x_max'], spacing['y_max'], spacing['z_max']))
size = np.array((spacing['x_out'], spacing['y_out'], spacing['z_out']))

for lobe_idx in range(1, 6):
    with open(f'/home/anna/Desktop/MA/ATM_data/{scan}/lobe_markers/{scan}_lobe{lobe_idx}.mrk.json') as f:
        lobe = json.load(f)
    points = lobe['markups'][0]['controlPoints']
    roots = []
    for i in range(len(points)//2):
        point1 = np.array(points[2*i]['position'])
        point2 = np.array(points[2*i+1]['position'])
        position = (point1-p_min) / (p_max-p_min) # normalizing to [0, 1]
        position *= size
        direction = (point2-point1) / np.linalg.norm(point2-point1)
        roots.append({
            'name': points[2*i]['label'].split('-')[0],
            'position': position.tolist(),
            'direction': direction.tolist()
        })
    lobes[f'lobe{lobe_idx}'] = roots
roots_dir = {
    'lobes': lobes
}
with open(out_file, 'w') as f:
    yaml.dump(lobes, f, default_flow_style=False, sort_keys=False)